## Robustness check

In order to assess the robustness of the results derived in sell_all_portfolio_simulation.ipynb, this notebook is implemented.   
Over a span of xxx years, xxx companies will be randomly drawn and a Portfolio simulationg following the LLM recommendations, as well as the
analyst recommendations will be run. The simulations' results will be collected and compared to enable a robust check of the previously obtained results.

In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from functions.faster_portfolio_simulation_class import PortfolioSimulation_fast
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [17]:
# Stock prices
stock_prices = pd.read_csv("../data/sp1500_monthly_prices.csv", dtype={"cik": str})
# Risk free rate df
risk_free_rate_df = pd.read_csv("../data/us3mt_yield_curve.csv")

- Reading in recommendations

In [18]:
# Read in CIK as string, so that leading zeros are preserved
analyst_ratings = pd.read_csv("../data/sp1500_sell_side_recommendations_ffilled.csv", dtype={"cik": str})
analyst_ratings.head()

,cik,date,mean_rating,rating,security
0,0000001750,2000-01,1.71429,buy,AAR CORP.
1,0000001750,2000-02,1.71429,buy,AAR CORP.
2,0000001750,2000-03,1.71429,buy,AAR CORP.
3,0000001750,2000-04,1.50000,strong buy,AAR CORP.
4,0000001750,2000-05,1.50000,strong buy,AAR CORP.


In [19]:
# LLM recommendations with only most recent financial statements
llm_recommendations1 = pd.read_csv("../data/ciklist1_ratings_with_most_recent_quarters.csv", dtype={"cik": str})
llm_recommendations2 = pd.read_csv("../data/ciklist2_ratings_with_most_recent_quarters.csv", dtype={"cik": str})
llm_recommendations3 = pd.read_csv("../data/ciklist3_ratings_with_most_recent_quarters.csv", dtype={"cik": str})
llm_recommendations4 = pd.read_csv("../data/missing_ratings.csv", dtype={"cik": str})

# Combine into one DataFrame
llm_recommendations = pd.concat([llm_recommendations1, llm_recommendations2, llm_recommendations3, llm_recommendations4], ignore_index=True)
llm_recommendations.head()

,cik,date,rating
0,0001497645,2010-03-31,sell
1,0001497645,2010-06-30,sell
2,0001497645,2010-09-30,sell
3,0001497645,2010-12-31,sell
4,0001497645,2011-03-31,sell


---
### Preprocessing

- Copied function to preprocess signals

In [20]:
def extract_signal(text):
    # Normalize text: lowercase, remove punctuation, remove extra spaces
    text_clean = ''.join(c for c in text.lower())
    
    # Map strong signals to regular ones
    if 'strong buy' in text_clean:
        return 'buy'
    if 'strong sell' in text_clean:
        return 'sell'
    if 'buy' in text_clean:
        return 'buy'
    if 'sell' in text_clean:
        return 'sell'
    if 'hold' in text_clean:
        return 'hold'
    return None  


In [21]:
 # Apply function
llm_recommendations["action"] = llm_recommendations["rating"].apply(extract_signal)
analyst_ratings["action"] = analyst_ratings["rating"].apply(extract_signal)

In [22]:
# Convert date to period format
llm_recommendations["date"] = pd.to_datetime(llm_recommendations["date"]).dt.to_period("M").astype(str)
analyst_ratings["date"] = pd.to_datetime(analyst_ratings["date"]).dt.to_period("M").astype(str)

# Drop duplicates
llm_recommendations.drop_duplicates(subset=["cik", "date"], inplace=True)
analyst_ratings.drop_duplicates(subset=["cik", "date"], inplace=True)

len(llm_recommendations), len(analyst_ratings)

(120860, 355218)

In [23]:
# Subsetting unique CIK and date combinations from both datasets
llm_ciks_and_dates = llm_recommendations[["cik", "date"]].drop_duplicates()
analyst_ciks_and_dates = analyst_ratings[["cik", "date"]].drop_duplicates() # drop duplicates may be double, but just to be safe

# Determine overlap through merge
shared_ciks_and_dates = pd.merge(llm_ciks_and_dates, analyst_ciks_and_dates, on=["cik", "date"], how="inner")

# Determine unique ciks, dates and min/max year
ciks = shared_ciks_and_dates["cik"].unique()
dates = shared_ciks_and_dates["date"].unique()
min_period = pd.to_datetime(dates).to_period("M").min()
max_period = pd.to_datetime(dates).to_period("M").max()

In [24]:
# Only keep CIK date combinations that are in both datasets
llm_recommendations_final = pd.merge(
    llm_recommendations, shared_ciks_and_dates, on=["cik", "date"], how="inner"
)
analyst_ratings_final = pd.merge(
    analyst_ratings, shared_ciks_and_dates, on=["cik", "date"], how="inner"
)

# Construct "temp"-date column still as period for filtering later on
llm_recommendations_final["date_tmp"] = pd.to_datetime(llm_recommendations_final["date"]).dt.to_period("M")
analyst_ratings_final["date_tmp"] = pd.to_datetime(analyst_ratings_final["date"]).dt.to_period("M")

# Check format
len(llm_recommendations_final), len(analyst_ratings_final)

(113301, 113301)

---
### Actual robustness check

In [32]:
# Number of companies to draw
sample_size = 10
# Number of months to regard
timespan = 12
# Number of simulations
iterations = 10

In [33]:
llm_recommendations_final["cik"].nunique(), analyst_ratings_final["cik"].nunique()

(1483, 1483)

In [38]:
# Empty list to store results
robustness_results = []

for i in tqdm(range(iterations), desc = "Simulating trading strategies"):
    # Set seed for reproducibility
    np.random.seed(i)
    # Random selection of CIKs
    robustness_ciks = np.random.choice(ciks, size=sample_size, replace=False)

    # Convert dates to periods for sampling (and comparison)
    periods = pd.to_datetime(dates).to_period("M")

    # Valid start dates = those with +10y still in range
    valid_starts = [p for p in periods if p + timespan <= periods.max()]  # 120 months = 10 years

    # Random start
    robustness_start = np.random.choice(valid_starts)
    robustness_end = robustness_start + timespan  # timespan many months later

    # Filter both recommendation dfs for valid dates
    llm_recommendations_simulation = llm_recommendations_final[
        (llm_recommendations_final["date_tmp"] >= robustness_start) &
        (llm_recommendations_final["date_tmp"] <= robustness_end)
    ].drop(columns="date_tmp")

    analyst_ratings_simulation = analyst_ratings_final[
        (analyst_ratings_final["date_tmp"] >= robustness_start) &
        (analyst_ratings_final["date_tmp"] <= robustness_end)
    ].drop(columns="date_tmp")

    # Filter both dfs for valid CIKs
    llm_recommendations_simulation = llm_recommendations_simulation[
        llm_recommendations_simulation["cik"].isin(robustness_ciks)
    ]
    analyst_ratings_simulation = analyst_ratings_simulation[
        analyst_ratings_simulation["cik"].isin(robustness_ciks)
    ]

    print(f"Simulation {i+1}/{iterations}: {robustness_start} to {robustness_end}, Number of CIKs (analyst): {analyst_ratings_simulation['cik'].nunique()}, Number of CIKs (LLM): {llm_recommendations_simulation['cik'].nunique()}")
    # Draft simulations
    start_capital = 1000000

    # Initialize LLM portfolio simulation
    llm_sim = PortfolioSimulation_fast(initial_capital=start_capital)
    llm_sim.partial_shares = True

    # Load data
    llm_sim.load_dataframes(stock_prices, llm_recommendations_simulation, risk_free_rate_df)
    # Run simulation
    llm_sim.simulate_trading()


    # Initialize Analyst simulation
    analyst_sim = PortfolioSimulation_fast(initial_capital=start_capital)
    analyst_sim.partial_shares = True

    # Load data
    analyst_sim.load_dataframes(stock_prices, analyst_ratings_simulation, risk_free_rate_df)
    # Run simulation
    analyst_sim.simulate_trading()

    # Compute monthly returns
    llm_returns = llm_sim.calculate_monthly_returns()
    analyst_returns = analyst_sim.calculate_monthly_returns()

    # Compute statistics
    llm_stats = llm_sim.portfolio_statistics(monthly_returns =  llm_returns)
    analyst_stats = analyst_sim.portfolio_statistics(monthly_returns = analyst_returns)

    # Grab most important stats
    llm_sr = llm_stats["Annualized Sharpe Ratio"]
    llm_mean_ret = llm_stats["Annualized mean return"]
    llm_final_val = llm_stats["Final Portfolio value (normalized)"]

    analyst_sr = analyst_stats["Annualized Sharpe Ratio"]
    analyst_mean_ret = analyst_stats["Annualized mean return"]
    analyst_final_val = analyst_stats["Final Portfolio value (normalized)"]

    # Append simulation results to list
    robustness_results.append({
        "simulation": i + 1,
        "start_date": robustness_start,
        "end_date": robustness_end,
        "no_companies": sample_size,
        "llm_sr": llm_sr,
        "llm_mean_ret": llm_mean_ret,
        "llm_final_val": llm_final_val,
        "analyst_sr": analyst_sr,
        "analyst_mean_ret": analyst_mean_ret,
        "analyst_final_val": analyst_final_val
    })


# Once all simulations are finished, compile results into a DataFrame
robustness_results_df = pd.DataFrame(robustness_results)

Simulating trading strategies:   0%|          | 0/10 [00:00<?, ?it/s]

Simulation 1/10: 2005-02 to 2006-02, Number of CIKs (analyst): 9, Number of CIKs (LLM): 9


Simulating trading strategies:  10%|█         | 1/10 [00:02<00:19,  2.20s/it]

Simulation 2/10: 2012-04 to 2013-04, Number of CIKs (analyst): 7, Number of CIKs (LLM): 7


Simulating trading strategies:  20%|██        | 2/10 [00:03<00:13,  1.74s/it]

...

Simulating trading strategies:  80%|████████  | 8/10 [00:11<00:02,  1.31s/it]

Simulation 9/10: 2015-01 to 2016-01, Number of CIKs (analyst): 7, Number of CIKs (LLM): 7


Simulating trading strategies:  90%|█████████ | 9/10 [00:13<00:01,  1.50s/it]

Simulation 10/10: 2021-02 to 2022-02, Number of CIKs (analyst): 10, Number of CIKs (LLM): 10


Simulating trading strategies: 100%|██████████| 10/10 [00:16<00:00,  1.67s/it]


In [39]:
robustness_results_df

,simulation,start_date,end_date,no_companies,llm_sr,llm_mean_ret,llm_final_val,analyst_sr,analyst_mean_ret,analyst_final_val
0,1,2005-02,2006-02,10,0.707634,0.190810,1.190810,0.469475,0.126910,1.126910
1,2,2012-04,2013-04,10,2.500325,0.681942,1.476924,3.080813,0.728609,1.507553
2,3,2013-01,2014-01,10,1.461479,0.272721,1.222581,0.435613,0.093317,1.077180
3,4,2007-04,2008-04,10,-2.196743,-0.123811,0.876189,-0.863899,-0.084643,0.915357
4,5,2019-02,2020-02,10,-0.822155,-0.040219,0.969682,1.507989,0.071033,1.052815
5,6,2007-06,2008-06,10,-0.916717,-0.139733,0.860267,-1.186990,-0.243523,0.756477
6,7,2023-06,2024-06,10,-0.071423,0.009202,1.008432,0.867443,0.320789,1.290518
7,8,2008-07,2009-07,10,0.639651,0.086132,1.078679,-0.658390,-0.624822,0.407116
8,9,2015-01,2016-01,10,0.650639,0.128181,1.128181,0.536675,0.106989,1.106989
9,10,2021-02,2022-02,10,0.271135,0.035209,1.029256,-0.104569,-0.032537,0.972811


---
##### Version to track all computation times

In [ ]:
# Empty list to store results
robustness_results = []

for i in range(iterations):

    # Random selection of CIKs
    robustness_ciks = np.random.choice(ciks, size=sample_size, replace=False)

    # Convert dates to periods for sampling (and comparison)
    periods = pd.to_datetime(dates).to_period("M")

    # Valid start dates = those with +10y still in range
    valid_starts = [p for p in periods if p + timespan <= periods.max()]  # 120 months = 10 years

    # Random start
    robustness_start = np.random.choice(valid_starts)
    robustness_end = robustness_start + timespan  # timespan many months later

    # Filter both recommendation dfs for valid dates
    llm_recommendations_simulation = llm_recommendations_final[
        (llm_recommendations_final["date_tmp"] >= robustness_start) &
        (llm_recommendations_final["date_tmp"] <= robustness_end)
    ].drop(columns="date_tmp")

    analyst_ratings_simulation = analyst_ratings_final[
        (analyst_ratings_final["date_tmp"] >= robustness_start) &
        (analyst_ratings_final["date_tmp"] <= robustness_end)
    ].drop(columns="date_tmp")

    # Filter both dfs for valid CIKs
    llm_recommendations_simulation = llm_recommendations_simulation[
        llm_recommendations_simulation["cik"].isin(robustness_ciks)
    ]
    analyst_ratings_simulation = analyst_ratings_simulation[
        analyst_ratings_simulation["cik"].isin(robustness_ciks)
    ]

    print(f"Simulation {i+1}/{iterations}: {robustness_start} to {robustness_end}, Number of CIKs: {analyst_ratings_simulation['cik'].nunique()}")
    # Draft simulations
    start_capital = 1000000

    # Initialize LLM portfolio simulation
    llm_sim = PortfolioSimulation_fast(initial_capital=start_capital)
    llm_sim.partial_shares = True

    # Load data
    llm_sim.load_dataframes(stock_prices, llm_recommendations_simulation, risk_free_rate_df)
    # Run simulation
    print(f"Running LLM simulation...")
    start_time = datetime.now()
    llm_sim.simulate_trading()
    end_time = datetime.now()
    elapsed = end_time - start_time
    # Convert to minutes and seconds
    minutes = elapsed.seconds // 60
    seconds = elapsed.seconds % 60
    print(f"LLM simulation completed in {minutes} min {seconds} sec")

    # Initialize Analyst simulation
    analyst_sim = PortfolioSimulation_fast(initial_capital=start_capital)
    analyst_sim.partial_shares = True

    # Load data
    analyst_sim.load_dataframes(stock_prices, analyst_ratings_simulation, risk_free_rate_df)
    # Run simulation
    print(f"Running Analyst simulation...")
    start_time = datetime.now()
    analyst_sim.simulate_trading()
    end_time = datetime.now()
    elapsed = end_time - start_time
    # Convert to minutes and seconds
    minutes = elapsed.seconds // 60
    seconds = elapsed.seconds % 60
   
    print(f"Analyst simulation completed in {minutes} min {seconds} sec")
    # Compute PF statistics for both
    print(f"Computing LLM returns...")
    start_time = datetime.now()
    llm_returns = llm_sim.calculate_monthly_returns()
    end_time = datetime.now()
    elapsed = end_time - start_time
    # Convert to minutes and seconds
    minutes = elapsed.seconds // 60
    seconds = elapsed.seconds % 60
    print(f"LLM returns computed in {minutes} min {seconds} sec")

    print(f"Computing Analyst returns...")
    start_time = datetime.now()
    analyst_returns = analyst_sim.calculate_monthly_returns()
    end_time = datetime.now()
    elapsed = end_time - start_time
    # Convert to minutes and seconds
    minutes = elapsed.seconds // 60
    seconds = elapsed.seconds % 60
    print(f"Analyst returns computed in {minutes} min {seconds} sec")

    # Compute statistics
    llm_stats = llm_sim.portfolio_statistics(monthly_returns =  llm_returns)
    analyst_stats = analyst_sim.portfolio_statistics(monthly_returns = analyst_returns)

    # Grab most important stats
    llm_sr = llm_stats["Annualized Sharpe Ratio"]
    llm_mean_ret = llm_stats["Annualized mean return"]
    llm_final_val = llm_stats["Final Portfolio value (normalized)"]

    analyst_sr = analyst_stats["Annualized Sharpe Ratio"]
    analyst_mean_ret = analyst_stats["Annualized mean return"]
    analyst_final_val = analyst_stats["Final Portfolio value (normalized)"]

    # Append simulation results to list
    robustness_results.append({
        "simulation": i + 1,
        "start_date": robustness_start,
        "end_date": robustness_end,
        "no_companies": sample_size,
        "llm_sr": llm_sr,
        "llm_mean_ret": llm_mean_ret,
        "llm_final_val": llm_final_val,
        "analyst_sr": analyst_sr,
        "analyst_mean_ret": analyst_mean_ret,
        "analyst_final_val": analyst_final_val
    })


# Once all simulations are finished, compile results into a DataFrame
robustness_results_df = pd.DataFrame(robustness_results)

In [13]:
robustness_results_df.to_csv("../data/robustness_check_results1.csv", index=False)